In [ ]:
# ==============================================================================
# 📓 学生生活効率化！課題・スケジュール管理ツール (最終完成版)
# ==============================================================================
# このツールは、日々の課題を管理し、Googleカレンダー/Tasksと連携、
# LINEでリマインドすることで、タスクの抜け漏れを防ぐことを目的としています。

# ==============================================================================
# ## 1. 準備：ライブラリのインストールとインポート
# ==============================================================================
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib requests

import json
import os
import requests
from datetime import datetime, timedelta, date

from google.colab import userdata
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build

# ==============================================================================
# ## 2. 秘密情報の設定
# ==============================================================================
try:
    LINE_CHANNEL_ACCESS_TOKEN = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
    LINE_USER_ID = userdata.get('LINE_USER_ID')
    GOOGLE_CLIENT_SECRET_JSON_STR = userdata.get('GOOGLE_CLIENT_SECRET_JSON')
    print("✅ 秘密情報の読み込みに成功しました。")
except Exception as e:
    print(f"❌ 秘密情報の読み込みに失敗しました。Colabのシークレット（🔑）が正しく設定されているか確認してください。エラー: {e}")

# ==============================================================================
# ## 3. Googleへの認証
# ==============================================================================
creds = None
TOKEN_PATH = 'token.json'
SCOPES = ['https://www.googleapis.com/auth/calendar', 'https://www.googleapis.com/auth/tasks']
if os.path.exists(TOKEN_PATH):
    creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        creds_file_path = 'client_secret.json'
        with open(creds_file_path, 'w') as f: f.write(GOOGLE_CLIENT_SECRET_JSON_STR)
        flow = Flow.from_client_secrets_file(
            creds_file_path, scopes=SCOPES, redirect_uri='urn:ietf:wg:oauth:2.0:oob')
        auth_url, _ = flow.authorization_url(prompt='consent')
        print('\n以下のURLにアクセスして、認証を完了してください:')
        print(auth_url)
        code = input('ブラウザに表示された認証コードをここに貼り付けてください: ')
        flow.fetch_token(code=code)
        creds = flow.credentials
    with open(TOKEN_PATH, 'w') as token: token.write(creds.to_json())
print("\n✅ Googleへの認証に成功しました！")
service_calendar = build('calendar', 'v3', credentials=creds)
service_tasks = build('tasks', 'v1', credentials=creds)

# ==============================================================================
# ## 4. 基本機能の定義 (最終改善版)
# ==============================================================================
tasks = [] # タスク管理リスト

# --- 4.1 ユーザー入力支援機能 (★★改善点1★★) ---
def get_datetime_from_user(prompt_message):
    """対話形式でユーザーから日時を取得し、datetimeオブジェクトを返す"""
    print(prompt_message)

    # 日付の取得
    date_str = input("日付を入力してください (例: 10/25 や 25 のように入力。今日の場合は Enter): ")
    if not date_str:
        target_date = date.today()
    else:
        try:
            month, day = map(int, date_str.split('/'))
            target_date = date(date.today().year, month, day)
        except ValueError:
            target_date = date(date.today().year, date.today().month, int(date_str))

    # 時間の取得
    time_str = input("時間を入力してください (例: 14:30 や 9:00 のように入力): ")
    hour, minute = map(int, time_str.split(':'))

    return datetime(target_date.year, target_date.month, target_date.day, hour, minute)

# --- 4.2 ローカルタスク管理 ---
def add_task(task_name, deadline_dt):
  task = {"task_name": task_name, "deadline": deadline_dt.strftime('%Y-%m-%d %H:%M:%S')}
  tasks.append(task)
  print(f"📝 ローカルにタスクを追加しました: {task_name}")

def view_tasks():
  print("\n--- 📝 現在のタスク一覧 ---")
  if not tasks:
    print("タスクはありません。")
    return
  for i, task in enumerate(tasks, 1):
    print(f"{i}. 課題名: {task['task_name']}, 締め切り: {task['deadline']}")

# --- 4.3 外部サービス連携 ---
def add_event_to_calendar(task_name, start_dt, end_dt):
  event = {
    'summary': task_name,
    'start': {'dateTime': start_dt.isoformat(), 'timeZone': 'Asia/Tokyo'},
    'end': {'dateTime': end_dt.isoformat(), 'timeZone': 'Asia/Tokyo'},
  }
  created_event = service_calendar.events().insert(calendarId='primary', body=event).execute()
  print(f"📅 Googleカレンダーに予定を追加しました: {created_event.get('htmlLink')}")

def add_google_task_with_deadline(task_list_id, task_name, deadline_dt):
  due_time = deadline_dt.astimezone().isoformat()
  task_body = {'title': task_name, 'due': due_time}
  result = service_tasks.tasks().insert(tasklist=task_list_id, body=task_body).execute()
  print(f"✅ Google ToDoリストに期日付きでタスクを追加しました: {result['title']}")

def send_line_message(message):
  url, headers = 'https://api.line.me/v2/bot/message/push', {'Content-Type': 'application/json', 'Authorization': f'Bearer {LINE_CHANNEL_ACCESS_TOKEN}'}
  data = {'to': LINE_USER_ID, 'messages': [{'type': 'text', 'text': message}]}
  response = requests.post(url, headers=headers, data=json.dumps(data))
  if response.status_code == 200: print(f"📱 LINEにメッセージを送信しました。")
  else: print(f"❌ LINEメッセージ送信に失敗しました: {response.text}")

# --- 4.4 リマインダー機能 (★★改善点2★★) ---
def check_deadlines_and_notify():
  print("\n--- ⏰ デッドラインチェック開始 ---")
  now = datetime.now()
  found_task = False
  for task in tasks:
    try:
        deadline_dt = datetime.strptime(task['deadline'], '%Y-%m-%d %H:%M:%S')
        time_diff = deadline_dt - now
        # 締め切りが「未来」であり、かつ「24時間以内（24時間ちょうどを含む）」かどうかをチェック
        if timedelta(days=0) < time_diff <= timedelta(hours=24): # ★★★ バグ修正: < を <= に変更 ★★★
          found_task = True
          message = (f"🔔 リマインダー 🔔\n「{task['task_name']}」の締め切りが近づいています！\n締め切り: {task['deadline']}")
          send_line_message(message)
    except ValueError:
        continue
  if not found_task:
    print("24時間以内に締め切りのタスクはありませんでした。")

# ==============================================================================
# ## 5. ツールの対話的実行 (最終改善版)
# ==============================================================================
while True:
    print("\n----------------------------------------")
    print("何をしますか？ 番号を入力してください。")
    print("1: タスクを新規追加")
    print("2: 現在のタスク一覧を表示")
    print("3: タスクをGoogleサービスに連携")
    print("4: 締め切りが近いタスクをLINEに通知")
    print("9: ツールを終了")
    choice = input("> ")

    if choice == '1':
        task_name = input("課題名を入力してください: ")
        try:
            deadline_dt = get_datetime_from_user("締め切り/開始日時を教えてください。")
            add_task(task_name, deadline_dt)
        except Exception as e:
            print(f"❌ 入力が正しくありません。最初からやり直してください。エラー: {e}")

    elif choice == '2':
        view_tasks()

    elif choice == '3':
        view_tasks()
        if not tasks: continue
        try:
            task_num = int(input("Googleに連携するタスクの番号を入力してください: "))
            task_index = task_num - 1
            target_task = tasks[task_index]

            service_choice = input("どちらに登録しますか？ (1: カレンダー, 2: ToDoリスト): ")
            if service_choice == '1':
                start_dt = datetime.strptime(target_task['deadline'], '%Y-%m-%d %H:%M:%S')
                end_dt = get_datetime_from_user(f"開始日時は {target_task['deadline']} です。終了日時を教えてください。")
                add_event_to_calendar(target_task['task_name'], start_dt, end_dt)
            elif service_choice == '2':
                YOUR_TASK_LIST_ID = 'MDgzNTMxMjY0MjU5MjY3MTg3MDE6MDow' # ★★ご自身のIDに書き換えてください★★
                deadline_dt = datetime.strptime(target_task['deadline'], '%Y-%m-%d %H:%M:%S')
                add_google_task_with_deadline(YOUR_TASK_LIST_ID, target_task['task_name'], deadline_dt)
            else:
                print("❌ 無効な選択です。")
        except Exception as e:
            print(f"❌ 処理中にエラーが発生しました。入力内容を確認してください。エラー: {e}")

    elif choice == '4':
        check_deadlines_and_notify()

    elif choice == '9':
        print("ツールを終了します。お疲れ様でした！")
        break
    else:
        print("❌ 無効な選択です。")